# Esame Laboratorio di Programmazione I
**Data: 07/02/2025**

Il file GlobalTemperatures.csv contiene la serie temporale delle temperature medie mensili globali, registrate a partire da Gennaio 1900 a Dicembre 2015. Il primo elemento di ogni riga rappresenta la data (colonna dt) mentre il secondo elemento (LandAverageTemperature) rappresenta la temperatura media globale per quel mese.

Vogliamo leggere questo tipo di dati e calcolare, dato un intervallo di anni, la differenza tra la temperatura media annuale e la media degli N anni precedenti.

## Svolgimento
Chiamate il file in cui scrivere il vostro codice `esame_matricola.py`. Commentate bene il codice descrivendo quello che fate!!!

### 1 (8 punti) Leggere i dati e processarli
Create la classe `CSVTimeSeriesFile`.
- La classe deve essere istanziata con il nome del file tramite la variabile `name`
- Deve avere un metodo `get_data()` che torni una lista di liste, dove il primo elemento delle liste annidate è la data ed il secondo la temperatura media mensile.

Questa classe si dovrà quindi poter usare così:
```python
time_series_file = CSVTimeSeriesFile(name='GlobalTemperatures.csv')
time_series = time_series_file.get_data()
```


In [ ]:
class ExamException(Exception):
    pass

class CSVTimeSeriesFile:
    def __init__(self, name):
        self.name = name
        # Check if file exists in __init__
        try:
            with open(self.name, 'r') as file:
                pass
        except:
            raise ExamException("Errore: impossibile aprire il file.")
    
    def get_data(self):
        time_series = []
        try:
            with open(self.name, 'r') as file:
                # Skip the header line
                next(file)
                for line in file:
                    elements = line.strip().split(',')
                    if len(elements) >= 2:
                        date = elements[0]
                        try:
                            # Check if value is valid (numeric and not negative)
                            value = float(elements[1])
                            if value >= 0:  # Only include non-negative values
                                time_series.append([date, value])
                        except ValueError:
                            # Skip non-numeric values
                            continue
        except Exception as e:
            raise ExamException(f"Errore nella lettura del file: {str(e)}")
        return time_series


[['1900-02-01', 11.19], ['1900-04-01', 8.51], ['1900-05-01', 7.23], ['1900-06-01', 6.36], ['1900-07-01', 5.63]]
1364



### 2 (14 punti) Calcolare la variazione delle temperature
Per calcolare la differenza tra la temperatura media annuale e la media degli N anni precedenti in un certo intervallo, dovete creare una funzione a sé stante (definita NON nella classe `CSVTimeSeriesFile` ma direttamente nel corpo principale del programma), di nome `compute_variations`, che ha come input la serie temporale, l'intervallo ed il parametro N della lunghezza della finestra e verrà usata così:

```python
compute_variations(time_series, first_year, last_year, N)
```

Dove `first_year` e `last_year` corrispondono agli estremi dell'intervallo di anni da considerare (inclusi) e dovrà ritornare in output (tramite un return) un dizionario, dove le chiavi sono gli anni presi in considerazione nell'intervallo ed i valori la differenza tra la temperatura media annuale di quell'anno e la media mobile degli N anni precedenti. Gli anni che possiamo calcolare dipendono dal parametro N. Quindi ad esempio, supponiamo di considerare l'intervallo 1900-1904 con N = 3.

La funzione dovrà:
- Raggruppare i dati per anno, per gli anni dell'intervallo:
  - 1900: [8.5, 9.2, 10.1, ...]
  - 1901: [8.7, 9.0, 9.8, ...]
  - ...
- Calcolare la media annuale per ciascun anno
- Calcolare la media basata sui 3 anni precedenti (disponibile solo a partire dal 1903 se no esco dall'intervallo preso in considerazione):
  
  Media Mobile 1903 = (Media 1900 + Media 1901 + Media 1902) / 3
  
  Media Mobile 1904 = (Media 1901 + Media 1902 + Media 1903) / 3

- Calcolare la differenza tra la temperatura media annuale e la media dei 3 anni precedenti (solo per 1903 e 1904):
  
  valore_1903 : Media annuale - Media mobile 1903
  2
  valore_1904 : Media annuale - Media mobile 1904

- Restituire un dizionario strutturato così:
  ```python
  {
    "1903": valore_1903,
    "1904": valore_1904
  }
  ```



In [81]:
def compute_variations(time_series, first_year, last_year, N):
    # Check if N is valid
    if N >= (last_year - first_year + 1):
        raise ExamException(f"N deve essere minore della lunghezza dell'intervallo ({last_year - first_year + 1})")
    
    # Group data by year
    data_by_year = {}
    for entry in time_series:
        date = entry[0]
        temp = entry[1]
        year = int(date.split('-')[0])
        if year not in data_by_year:
            data_by_year[year] = []
        data_by_year[year].append(temp)
    
    # Calculate annual averages
    annual_avgs = {}
    for year in range(first_year - N, last_year + 1):
        if year in data_by_year:
            annual_avgs[year] = sum(data_by_year[year]) / len(data_by_year[year])
    
    # Calculate variations (difference between annual average and moving average of N previous years)
    variations = {}
    for year in range(first_year + N, last_year + 1):
        if year in annual_avgs:
            # Calculate moving average of N previous years
            moving_avg = 0
            valid_prev_years = 0
            for prev_year in range(year - N, year):
                if prev_year in annual_avgs:
                    moving_avg += annual_avgs[prev_year]
                    valid_prev_years += 1
            
            if valid_prev_years > 0:
                moving_avg /= valid_prev_years
                variations[str(year)] = annual_avgs[year] - moving_avg
    
    return variations


# Funzione opzionale per la lode
def find_years_outside_range(time_series, min_temp, max_temp):
    """
    Data una serie temporale e un intervallo di temperatura [min_temp, max_temp],
    restituisce la lista degli anni che hanno almeno un mese con un valore fuori dall'intervallo.
    
    Args:
        time_series: Serie temporale come lista di liste [data, temperatura]
        min_temp: Temperatura minima dell'intervallo
        max_temp: Temperatura massima dell'intervallo
        
    Returns:
        Lista degli anni con almeno un mese fuori dall'intervallo di temperatura
    """
    # Controllo validità degli estremi dell'intervallo
    if not isinstance(min_temp, (int, float)) or not isinstance(max_temp, (int, float)):
        raise ExamException("Gli estremi dell'intervallo devono essere numerici")
    
    if min_temp > max_temp:
        raise ExamException("Il minimo dell'intervallo non può essere maggiore del massimo")
    
    # Dizionario per tracciare gli anni con mesi fuori intervallo
    years_outside_range = set()
    
    for entry in time_series:
        date = entry[0]
        temp = entry[1]
        
        try:
            year = int(date.split('-')[0])
            
            # Controllo se la temperatura è fuori dall'intervallo
            if temp < min_temp or temp > max_temp:
                years_outside_range.add(year)
        except (ValueError, IndexError):
            # Salto date malformate
            continue
    
    # Converto il set in una lista ordinata
    return sorted(list(years_outside_range))

# Test con un esempio
time_series_file = CSVTimeSeriesFile(name='GlobalTemperatures.csv')
time_series = time_series_file.get_data()

print("Prime 5 righe del time series:")
print(time_series[:5])

# Calcolo variazioni per l'intervallo 2000-2015 con N=3
result = compute_variations(time_series, 2000, 2015, 3)
print("\nRisultato compute_variations (2000-2015, N=3):")
for year, variation in result.items():
    print(f'"{year}": {variation:.6f}')

# Test della funzione opzionale per la lode
try:
    # Esempio: trovare anni con temperature fuori dall'intervallo [8.0, 10.0]
    outside_range_years = find_years_outside_range(time_series, 8.0, 10.0)
    print("\nAnni con almeno un mese con temperatura fuori dall'intervallo [8.0, 10.0]:")
    print(outside_range_years)
except ExamException as e:
    print(f"Errore: {str(e)}")

# Test con intervallo invalido
try:
    invalid_range = find_years_outside_range(time_series, 10.0, 8.0)
    print(invalid_range)  # Non dovrebbe arrivare qui
except ExamException as e:
    print(f"\nTest con intervallo invalido - Errore atteso: {str(e)}")



Prime 5 righe del time series:
[['1900-02-01', 11.19], ['1900-04-01', 8.51], ['1900-05-01', 7.23], ['1900-06-01', 6.36], ['1900-07-01', 5.63]]

Risultato compute_variations (2000-2015, N=3):
"2003": 0.018444
"2004": -0.056944
"2005": 0.000556
"2006": 0.069722
"2007": 0.148056
"2008": 0.451000
"2009": -0.195056
"2010": -0.107833
"2011": -0.045611
"2012": -0.069722
"2013": -0.163333
"2014": -0.059444
"2015": 0.192222

Anni con almeno un mese con temperatura fuori dall'intervallo [8.0, 10.0]:
[1900, 1901, 1902, 1903, 1904, 1905, 1906, 1907, 1908, 1909, 1910, 1911, 1912, 1913, 1914, 1915, 1916, 1917, 1918, 1919, 1920, 1921, 1922, 1923, 1924, 1925, 1926, 1927, 1928, 1929, 1930, 1931, 1932, 1933, 1934, 1935, 1936, 1937, 1938, 1939, 1940, 1941, 1942, 1943, 1944, 1945, 1946, 1947, 1948, 1949, 1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 

### 3 (8 punti) Eccezioni e Controllo dell'input
Le eccezioni da alzare in caso di input non corretti o casi limite devono essere istanze di una specifica classe `ExamException`, che dovete definire nel codice come segue, senza modifica alcuna (copia-incollate le due righe):

```python
class ExamException(Exception):
    pass
```

... e che poi userete come una normale eccezione, ad esempio:

```python
raise ExamException("Errore: impossibile aprire il file.")
```

Per l'esame dovete considerare che:
- **(2 punti)** La classe CSVTimeSeriesFile deve controllare l'esistenza del file nell'`__init__()` (non in `get_data`) e, nel caso il file non esista o non sia leggibile, alza un'eccezione.
- **(2 punti)** I valori che leggete dal file CSV sono da aspettarsi di tipo float, un valore non numerico, oppure vuoto o nullo o negativo non deve essere accettato, ma tutto deve procedere comunque senza alzare eccezioni.
- **(2 punti)** possono mancare misurazioni di mesi. Se mancano misurazioni per uno o più mesi, il valore medio delle temperatura andrà calcolato sul numero di mesi per cui abbiamo le misurazioni.
- **(2 punti)** Il parametro della finestra N deve essere strettamente minore della lunghezza dell'intervallo considerato altrimenti non possiamo calcolare nessun valore, nel caso non sia così alzare un'eccezione.

## Parte opzionale (per la Lode, solo se tutto il resto è giusto)
Aggiungere una funzione che, dato un intervallo di temperatura, mi calcola gli anni che hanno almeno un mese con un valore fuori dall'intervallo. Considerare le eccezioni per l'inserimento sbagliato degli estremi dell'intervallo.